In [59]:
# Importing required libraries
import numpy as np
import pandas as pd
import librosa
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [36]:
def read_audio_files(path, sample_rate=44100):
    audio_files = []
    for filename in os.listdir(path):
        if filename.endswith('.wav'):
            file_path = os.path.join(path, filename)
            sample_rate, data = wavfile.read(file_path)
            audio_files.append(data)
    return audio_files

In [37]:
def coarse_grain(data, level):
    data = np.array(data)
    coeffs = wavedec(data, 'haar', level=level)
    coeffs[-1] = np.zeros_like(coeffs[-1])
    coeffs[-2] = np.zeros_like(coeffs[-2])
    coeffs[-3] = np.zeros_like(coeffs[-3])
    coeffs[0] = np.zeros_like(coeffs[0])
    return np.concatenate(coeffs)

In [38]:
def hjorth_desc(data):
    first_deriv = np.diff(data)
    second_deriv = np.diff(data, 2)
    activity = np.var(data)
    mobility = np.sqrt(np.var(first_deriv) / activity)
    complexity = np.sqrt(np.var(second_deriv) / np.var(first_deriv))
    return [activity, mobility, complexity]

In [53]:
def extract_features(audio_files, labels, coarse_grain_level):
    features = []
    for i, audio_file in enumerate(audio_files):
        print(f'Processing audio file {i+1}/{len(audio_files)}')
        # Resample to 22050 Hz to reduce computation time
        audio_file = resample(audio_file, 22050)
        # Filter out frequencies above 5000 Hz to focus on heart sounds
        b, a = butter(4, 5000, 'low', fs=22050)
        audio_file = filtfilt(b, a, audio_file)
        # Split into 1-second chunks
        chunks = make_chunks(audio_file, 22050)
        for chunk in chunks:
            features.append(coarse_grain(chunk, coarse_grain_level))
            features[-1] = np.concatenate((features[-1], hjorth_desc(chunk)))
    features = np.array(features)
    labels = np.array(labels)
    return features, labels

In [61]:
# Defining function to create dataset
def create_dataset(path):
    features = []
    labels = []
    for label in os.listdir(path): # Loop over all labels
        label_path = os.path.join(path, label)
        for file in os.listdir(label_path): # Loop over all files in label directory
            file_path = os.path.join(label_path, file)
            feature = extract_features(file_path) # Extract features
            features.append(feature)
            labels.append(label) # Add label to list
    return np.array(features), np.array(labels)

In [56]:
normal_path = 'dataset/training-a/wav copy/normal'
abnormal_path = 'dataset/training-a/wav copy/abnormal'
normal_files = read_audio_files(normal_path)
abnormal_files = read_audio_files(abnormal_path)
normal_labels = np.zeros(len(normal_files))
abnormal_labels = np.ones(len(abnormal_files))
all_files = np.concatenate((normal_files, abnormal_files))
all_labels = np.concatenate((normal_labels, abnormal_labels))


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [57]:
features, labels = create_dataset(path)
features_scaled = scaler.fit_transform(features)
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)

Processing audio file 1/409
Processing audio file 2/409
Processing audio file 3/409
Processing audio file 4/409
Processing audio file 5/409
Processing audio file 6/409
Processing audio file 7/409
Processing audio file 8/409
Processing audio file 9/409
Processing audio file 10/409
Processing audio file 11/409
Processing audio file 12/409
Processing audio file 13/409
Processing audio file 14/409
Processing audio file 15/409
Processing audio file 16/409
Processing audio file 17/409
Processing audio file 18/409
Processing audio file 19/409
Processing audio file 20/409
Processing audio file 21/409
Processing audio file 22/409
Processing audio file 23/409
Processing audio file 24/409
Processing audio file 25/409
Processing audio file 26/409
Processing audio file 27/409
Processing audio file 28/409
Processing audio file 29/409
Processing audio file 30/409
Processing audio file 31/409
Processing audio file 32/409
Processing audio file 33/409
Processing audio file 34/409
Processing audio file 3

In [72]:
features, labels = extract_features(all_files, all_labels, coarse_grain_level=3)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)


Processing audio file 1/409
Processing audio file 2/409
Processing audio file 3/409
Processing audio file 4/409
Processing audio file 5/409
Processing audio file 6/409
Processing audio file 7/409
Processing audio file 8/409
Processing audio file 9/409
Processing audio file 10/409
Processing audio file 11/409
Processing audio file 12/409
Processing audio file 13/409
Processing audio file 14/409
Processing audio file 15/409
Processing audio file 16/409
Processing audio file 17/409
Processing audio file 18/409
Processing audio file 19/409
Processing audio file 20/409
Processing audio file 21/409
Processing audio file 22/409
Processing audio file 23/409
Processing audio file 24/409
Processing audio file 25/409
Processing audio file 26/409
Processing audio file 27/409
Processing audio file 28/409
Processing audio file 29/409
Processing audio file 30/409
Processing audio file 31/409
Processing audio file 32/409
Processing audio file 33/409
Processing audio file 34/409
Processing audio file 3

In [73]:
clf = SVC(kernel='rbf', C=10, gamma=0.1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.83      0.16      0.27        31
         1.0       0.66      0.98      0.79        51

    accuracy                           0.67        82
   macro avg       0.75      0.57      0.53        82
weighted avg       0.72      0.67      0.59        82



In [ ]:
mnb = SVC(kernel='rbf', C=10, gamma=0.1)
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_test)
print(classification_report(y_test, y_pred))

In [71]:
path = 'dataset/training-a/wav copy/'
features, labels = create_dataset(path)

TypeError: extract_features() missing 2 required positional arguments: 'labels' and 'coarse_grain_level'

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [66]:
# Creating LSTM model
model = Sequential()
model.add(LSTM(units=64, input_shape=X_train.shape[1:], return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

# Compiling model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Predicting labels for test set
y_pred = model.predict(X_test)
y_pred = [1 if y>=0.5 else 0 for y in y_pred]

# Calculating accuracy score
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

ValueError: Input 0 of layer "lstm_2" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 22055)